#### Import required dependencies and load the API key

In [ ]:
import * as mod from "https://deno.land/std@0.213.0/dotenv/mod.ts";
import { 
    Document, 
    VectorStoreIndex, 
    SimpleDirectoryReader,
    RouterQueryEngine,
    OpenAIAgent,
    QueryEngineTool,
    FunctionTool,
} from "npm:llamaindex@0.1.18"
const keys = await mod.load({export:true}) // read API key from .env

Let's create 2 different data sources that answer different types of questions:
- The first is the same example as in lesson 1, an essay by Dan Abramov about his career.
- The second is the wikipedia page content about React.

**Note**: To access the data used in this lesson, go to `File` and click on `Open` 

In [ ]:
const documents1 = await new SimpleDirectoryReader().loadData({directoryPath: "./data"})
const index1 = await VectorStoreIndex.fromDocuments(documents1)
const queryEngine1 = index1.asQueryEngine()

In [ ]:
let response1 = await queryEngine1.query({query: "What did the author do in college?"})
console.log(response1.toString())

In [ ]:
const documents2 = await new SimpleDirectoryReader().loadData({directoryPath: "./data2"})
const index2 = await VectorStoreIndex.fromDocuments(documents2)
const queryEngine2 = index2.asQueryEngine()

In [ ]:
let response2 = await queryEngine2.query({query:"What is React?"})
console.log(response2.toString())

Create a Router Query Engine

Ask it a Question

In [ ]:
let response3 = await queryEngine.query({query: "What is React?"})
console.log(response3.toString())

In [ ]:
let response4 = await queryEngine.query({query: "What did Dan Abramov do in college?"})
console.log(response4.toString())

Define a Simple Function

In [ ]:
function sumNumbers({a,b}) {
  return a + b;
}

Explain the Function in JSON Structure

In [ ]:
const sumJSON = {
  type: "object",
  properties: {
    a: {
      type: "number",
      description: "The first number",
    },
    b: {
      type: "number",
      description: "The second number",
    },
  },
  required: ["a", "b"],
};

Make the new function into a Tool

In [ ]:
const sumFunctionTool = new FunctionTool(sumNumbers, {
  name: "sumNumbers",
  description: "Use this function to sum two numbers",
  parameters: sumJSON,
});

Make the Router Query Engine into a Tool

In [ ]:
const queryEngineTool = new QueryEngineTool({
    queryEngine: queryEngine,
    metadata: {
        name: "react_and_dan_abramov_engine",
        description: "A tool that can answer questions about Dan Abramov and React",
    },
});

Use the two tools

In [ ]:
const agent = new OpenAIAgent({
    tools: [queryEngineTool, sumFunctionTool],
    verbose: true
})

Ask it a question

In [ ]:
let response5 = await agent.chat({message:"What is React? Use a tool."})
console.log(response5.toString())

In [ ]:
let response6 = await agent.chat({message:"What is 501 + 5?"})
console.log(response6.toString())